In [1]:
import asyncio

### declare coro by async def and await

In [ ]:
async def main():
    print("hello")
    await asyncio.sleep(1)
    print("world")


await main()

hello
world


### run via asyncio.run(), await, or tasks

In [ ]:
# await a coro
import time


async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, "hello")  # non-concurrent
    await say_after(2, "world")
    print(f"finished at {time.strftime('%X')}")


# asyncio.run(main())
await main()

started at 19:27:19
hello
world
finished at 19:27:22


In [18]:
# tasks
async def main():
    task1 = asyncio.create_task(say_after(1, "hello"))
    task2 = asyncio.create_task(say_after(2, "world"))

    print(f"started at {time.strftime('%X')}")
    # await asyncio.gather(task1, task2)
    await task1
    await task2
    print(f"finished at {time.strftime('%X')}")


await main()

started at 18:48:16
hello
world
finished at 18:48:18


### awaitables - coro, task, fut

In [13]:
async def nested():
    return 42


async def main():
    print(await nested())


await main()

42


In [12]:
async def nested():
    return 42


async def main():
    task = asyncio.create_task(nested())
    res = await task
    print(res)


await main()

42


### create task via asyncio.create_task()

In [30]:
background_tasks = set()


async def some_coro(param):
    # Do something with param.
    await asyncio.sleep(2)
    print(param)


async def main():
    for i in range(10):
        task = asyncio.create_task(some_coro(param=i))

        # Add task to the set. This creates a strong reference.
        background_tasks.add(task)

        # await task # not concurrent

        # To prevent keeping references to finished tasks forever,
        # make each task remove its own reference from the set after
        # completion:
        # task.add_done_callback(background_tasks.discard)

    # await asyncio.gather(*background_tasks)

    for task in background_tasks:
        await task


await main()

0
1
2
3
4
5
6
7
8
9


### asyncio.sleep to suspend coro


In [25]:
import asyncio
import datetime


async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)  # block/pause curr coro, give back control


await display_date()

2025-08-25 19:22:50.244168
2025-08-25 19:22:51.245723
2025-08-25 19:22:52.247346
2025-08-25 19:22:53.248856
2025-08-25 19:22:54.250420


### use gather for concurrent runnig


In [ ]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f


async def main():
    # auto schedule coros as tasks and wait for completion
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)


await main()

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


### wait_for timeout


In [ ]:
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print("yay!")


async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)  # auto schedule coro as task
    except asyncio.TimeoutError:
        print("timeout!")


await main()

timeout!


### to thread is only good for io bound funcs (due to GIL)


In [ ]:
def blocking_io():
    print(f"start blocking_io at {time.strftime('%X')}")
    time.sleep(1)
    print(f"blocking_io complete at {time.strftime('%X')}")


async def main():
    print(f"started main at {time.strftime('%X')}")

    await asyncio.gather(
        asyncio.to_thread(
            blocking_io
        ),  # return coro, which is auto scheduled as task by gather
        asyncio.to_thread(blocking_io),
        asyncio.to_thread(blocking_io),
        asyncio.sleep(1),
    )

    print(f"finished main at {time.strftime('%X')}")


await main()

started main at 19:47:04
start blocking_io at 19:47:04
start blocking_io at 19:47:04
start blocking_io at 19:47:04
blocking_io complete at 19:47:05blocking_io complete at 19:47:05

blocking_io complete at 19:47:05
finished main at 19:47:05


### task cancel

In [33]:
async def cancel_me():
    print("cancel_me(): before sleep")

    try:
        # Wait for 1 hour
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print("cancel_me(): cancel sleep")
        raise
    finally:
        print("cancel_me(): after sleep")


async def main():
    # Create a "cancel_me" Task
    task = asyncio.create_task(cancel_me())

    # Wait for 1 second
    await asyncio.sleep(1)

    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print("main(): cancel_me is cancelled now")


await main()

cancel_me(): before sleep
cancel_me(): cancel sleep
cancel_me(): after sleep
main(): cancel_me is cancelled now
